In [3]:
import os
import io
import pandas as pd
import numpy as np
import spacy as sp
from spacy import displacy
from spacy.tokens import SpanGroup
pd.set_option('display.max_colwidth', None)

In [4]:
#initial setup
nlp = sp.load("en_core_web_sm")
fp = os.path.join('data', 'bike.csv')
cleaned_bike = pd.read_csv(fp)[["Response Id", "Original Answer"]]
cleaned_bike

,Response Id,Original Answer
0,https://mathanex.com/student_response/1e70e201-459b-4d5b-bc9a-c96009256ef6/,Giovanni has to work 4 more hours because 15+15=30 15+15=60 15+15=90+5 he has $1.95 and he has more money and he buys it and he will have 15 money left.
1,https://mathanex.com/student_response/dcbc3f21-c374-4f9e-b88c-7c68292ecb39/,"Giovanni needs $180 to buy a bike. he already has $105 saved up which means that he already has some of the money he needs. we need to find out how many hours he needs to work to get his bike if he gets 15 dollars per hour. So I decided to subtract 105 dollars from the 180 dollars because he already has that much saved up and we need to find out how many more hours he needs to work to get his bike. subtracting works here because we can get closer to how much money we need which can get us closer to how many hours are required subtracting is a math strategy that is used to see how much of something you have left by taking away an amount. So by subtracting the 105 dollars, we can get an answer of 75 dollars as our difference. I also checked this answer by adding the 105 dollars plus our difference which is 75 and got 180. However, they are asking how many hours and not how much money. He gets 15 dollars per hour so by dividing which is splitting up numbers into groups we can find out how many hours he needs to work in this case, we would need to divide by 15 because that is how much he earns per hour. This way every group of 15s is an hour. So I can find out how many hours he needs. Here I kept on adding 15 to find out how many 15s I needed to add to get to 75. This way I can find out that Giovanni needs to work 5 more hours which is how many groups of 15s until I get to 75."
2,https://mathanex.com/student_response/71ef5744-c88e-45bd-abb4-0f2a88c227a9/,How did i figured out how many more hours Giovanni has to work to have enough for the bike is 5 hours how do i know is how many are in the 15 is 1 hours
3,https://mathanex.com/student_response/88476d20-0825-48a6-9c38-7663e0e32b94/,Because I added 15 to 105 until I got enough for the bike.
4,https://mathanex.com/student_response/a53569e1-5806-4700-b242-a4c95f2120bb/,180/15=12
...,...,...
6951,https://mathanex.com/student_response/4e8eace5-03b7-472d-95cf-a3b04278f230/,NaN
6952,https://mathanex.com/student_response/f5cf858c-a6f3-42b4-9086-2dd72a323132/,15 + $180.
6953,https://mathanex.com/student_response/c85bac79-0e49-4c96-94f0-4ff99b0d34d8/,180-10=75
6954,https://mathanex.com/student_response/35fe50dd-bb58-4323-8b71-bf8a13150281/,NaN


In [5]:
def conjugates(text):
    en = sp.load('en_core_web_sm')

    doc = en(str(text))
    #deplacy.render(doc)

    seen = set() # keep track of covered words

    chunks = []
    for sent in doc.sents:
        heads = [cc for cc in sent.root.children if cc.dep_ == 'conj']

        for head in heads:
            words = [ww for ww in head.subtree]
            for word in words:
                seen.add(word)
            chunk = (' '.join([ww.text for ww in words]))
            chunks.append( (head.i, chunk) )

        unseen = [ww for ww in sent if ww not in seen]
        chunk = ' '.join([ww.text for ww in unseen])
        chunks.append( (sent.root.i, chunk) )

    chunks = sorted(chunks, key=lambda x: x[0])

    return [x[1] for x in chunks]

In [6]:
conj_df = cleaned_bike[0:10]
conj_df["Subsentence"] = conj_df.get('Original Answer').apply(conjugates)
conj_df = conj_df.explode("Subsentence")
conj_df[["Subsentence", "Original Answer"]]

/var/folders/y6/cb6w36g12jq8n132f5hc5m600000gn/T/ipykernel_38619/3691450937.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conj_df["Subsentence"] = conj_df.get('Original Answer').apply(conjugates)


,Subsentence,Original Answer
0,Giovanni has to work 4 more hours because 15 + 15=30 15 + 15=60 15 + 15=90 .,Giovanni has to work 4 more hours because 15+15=30 15+15=60 15+15=90+5 he has $1.95 and he has more money and he buys it and he will have 15 money left.
0,+ 5 he has $ 1.95 and he has more money and he buys it and he will have 15 money left,Giovanni has to work 4 more hours because 15+15=30 15+15=60 15+15=90+5 he has $1.95 and he has more money and he buys it and he will have 15 money left.
1,Giovanni needs $ 180 to buy a bike .,"Giovanni needs $180 to buy a bike. he already has $105 saved up which means that he already has some of the money he needs. we need to find out how many hours he needs to work to get his bike if he gets 15 dollars per hour. So I decided to subtract 105 dollars from the 180 dollars because he already has that much saved up and we need to find out how many more hours he needs to work to get his bike. subtracting works here because we can get closer to how much money we need which can get us closer to how many hours are required subtracting is a math strategy that is used to see how much of something you have left by taking away an amount. So by subtracting the 105 dollars, we can get an answer of 75 dollars as our difference. I also checked this answer by adding the 105 dollars plus our difference which is 75 and got 180. However, they are asking how many hours and not how much money. He gets 15 dollars per hour so by dividing which is splitting up numbers into groups we can find out how many hours he needs to work in this case, we would need to divide by 15 because that is how much he earns per hour. This way every group of 15s is an hour. So I can find out how many hours he needs. Here I kept on adding 15 to find out how many 15s I needed to add to get to 75. This way I can find out that Giovanni needs to work 5 more hours which is how many groups of 15s until I get to 75."
1,he already has $ 105 saved up which means that he already has some of the money he needs .,"Giovanni needs $180 to buy a bike. he already has $105 saved up which means that he already has some of the money he needs. we need to find out how many hours he needs to work to get his bike if he gets 15 dollars per hour. So I decided to subtract 105 dollars from the 180 dollars because he already has that much saved up and we need to find out how many more hours he needs to work to get his bike. subtracting works here because we can get closer to how much money we need which can get us closer to how many hours are required subtracting is a math strategy that is used to see how much of something you have left by taking away an amount. So by subtracting the 105 dollars, we can get an answer of 75 dollars as our difference. I also checked this answer by adding the 105 dollars plus our difference which is 75 and got 180. However, they are asking how many hours and not how much money. He gets 15 dollars per hour so by dividing which is splitting up numbers into groups we can find out how many hours he needs to work in this case, we would need to divide by 15 because that is how much he earns per hour. This way every group of 15s is an hour. So I can find out how many hours he needs. Here I kept on adding 15 to find out how many 15s I needed to add to get to 75. This way I can find out that Giovanni needs to work 5 more hours which is how many groups of 15s until I get to 75."
1,we need to find out how many hours he needs to work to get his bike if he gets 15 dollars per hour .,"Giovanni needs $180 to buy a bike. he already has $105 saved up which means that he already has some of the money he needs. we need to find out how many hours he needs to work to get his bike if he gets 15 dollars per hour. So I decided to subtract 105 dollars from the 180 dollars because he already has that much saved up and we need to find out how many more hours he needs to work to get his bike. subtracting works here because we can get closer to how much mon

In [7]:
doc2 = nlp(cleaned_bike["Original Answer"][1])
sents = doc2.sents
phrases = []
for sentence in sents:
    phrase = ""
    for i in sentence.root.children:
        if i.dep_ == "nsubj":
            for j in i.subtree:
                phrase += j.text + " "
            phrase += sentence.root.text + " "
        elif i.dep_ == "advcl":
            for j in i.subtree:
                phrases[-1] += " " + j.text
            phrases[-1] = phrases[-1].strip()
        elif i.dep_ == "punct":
            phrases[-1] += i.text
        else:
            for j in i.subtree:
                phrase += j.text + " "
            phrases.append(phrase.strip())
            phrase = ""
        
phrases

['Giovanni needs $ 180 to buy a bike.',
 'he has already',
 '$ 105 saved up which means that he already has some of the money he needs.',
 'we need to find out how many hours he needs to work to get his bike if he gets 15 dollars per hour.',
 'So',
 'I decided to subtract 105 dollars from the 180 dollars because he already has that much saved up and we need to find out how many more hours he needs to work to get his bike.',
 'subtracting works here because we can get closer to how much money we need which can get us closer to how many hours are required is subtracting is a math strategy that is used to see how much of something you have left by taking away an amount.',
 'So',
 'by subtracting the 105 dollars,',
 'we get can',
 'an answer of 75 dollars as our difference.',
 'I checked also',
 'this answer',
 'by adding the 105 dollars plus our difference which is 75 and got 180.',
 'However,',
 'they asking are',
 'how many hours and not how much money.',
 'He gets 15 dollars per hour s